In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/DATA304'

/content/drive/MyDrive/DATA304


In [40]:
import json

predictions_file_path = 'predictions_12b.jsonl'
eval_file_path = 'eval.jsonl'

def read_jsonl_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data

predictions_data = read_jsonl_file(predictions_file_path)
eval_data = read_jsonl_file(eval_file_path)

(predictions_data[:3], eval_data[:3])

([{'input': '제목: 플스5 디스크 판매, \n 상품 설명: 군대 가게 되어 판매합니다',
   'output': '[예측하기]\n제목과 상품 설명을 참고해서 이 상품이 플랫폼에 업로드되고 나서 지난 시간이 7일 이상일지 또는 7일 미만일지 예측해 줘.\n간단하게 생각하면 상품이 판매되는 데에 7일 미만이 걸릴지 또는 7일 이상이 걸릴지 예측하는 것과 비슷해.\n네가 출력해야 하는 것은 상품이 플랫폼에 업로드되고 나서 지난 시간이 7일 이상일지 또는 7일 미만일지 예측한 결과야.\n상품이 판매되는 데에 7일 미만이 걸릴 것 같으면 \'0\'이라고 답하면 되고 7일 이상이 걸릴 것 같으면 \'1\'이라고 답하면 돼.\n\n제목: 플스5 디스크 판매, \n 상품 설명: 군대 가게 되어 판매합니다\n예측 결과: {"7": "1'},
  {'input': '제목: 후드 목베개 2개, \n 상품 설명: 후드가 달려있는 목베게입니다.\n2개, 새것입니다. 사진찍으려고 비닐포장에서 꺼냈습니다.\n잘 때 주변을 가려주고 보온효과도 있어요.\n사진처럼 2가지 색상으로 사용 가능한 리버시블입니다.\n투썸플레이스, 스타벅스 기프티콘과 교환가능합니다.\n챗으로 문의주세여\n판매글 잘 읽어보시고 거래해주세요!\n거래 후 단순변심으로 인한 반품환불 안됩니다.\n책임감 있게 행동해주세요 :)\n비대면 거래 가능합니다\n문의는 문의라고 명시 후 챗 주세요!',
   'output': '[예측하기]\n제목과 상품 설명을 참고해서 이 상품이 플랫폼에 업로드되고 나서 지난 시간이 7일 이상일지 또는 7일 미만일지 예측해 줘.\n간단하게 생각하면 상품이 판매되는 데에 7일 미만이 걸릴지 또는 7일 이상이 걸릴지 예측하는 것과 비슷해.\n네가 출력해야 하는 것은 상품이 플랫폼에 업로드되고 나서 지난 시간이 7일 이상일지 또는 7일 미만일지 예측한 결과야.\n상품이 판매되는 데에 7일 미만이 걸릴 것 같으면 \'0\'이라고 답하면 되고 7일 이상이 걸릴 것 같으면 \'1\'이라고 답하면 돼

In [45]:
import re

def extract_predicted_text(text):
    return text.split("예측 결과:")[-1].strip()

def extract_predicted_result(output_text):
    output_text = extract_predicted_text(output_text)

    count_1s = len(re.findall(r"\b1\b", output_text)) + len(re.findall("7일 이상", output_text)) + len(re.findall("one", output_text))
    count_0s = len(re.findall(r"\b0\b", output_text)) + len(re.findall("7일 미만", output_text)) + len(re.findall("zero", output_text))
    return '1' if count_1s >= count_0s else '0'

predicted_results = [int(extract_predicted_result(item['output'])) for item in predictions_data]

actual_labels = [int(item['output']) for item in eval_data]

In [46]:
from sklearn.metrics import accuracy_score, f1_score

def calculate_metrics(predictions, actuals):
    accuracy = accuracy_score(actuals, predictions)
    f1 = f1_score(actuals, predictions)

    return {
        "accuracy": accuracy,
        "f1_score": f1
    }

metrics = calculate_metrics(predicted_results, actual_labels)

metrics

{'accuracy': 0.49514563106796117, 'f1_score': 0.606060606060606}